# I. Setup

In [5]:
# import
import pandas as pd
import plotly.express as px

In [2]:
# import dataset
data = pd.read_csv("data/dataset.csv", sep = ",")
data = data.drop("total_length", axis=1)
data.head()

,user,gender,country,level_of_education,course_id,nb_threads,nb_comments,corpus,delai_1er_post,grade,certificate_eligible
0,--37,NaN,NaN,NaN,course-v1:MinesTelecom+04036+session01,2,4,je ne sais pas non plus :/. je me suis trompée...,44,NaN,NaN
1,--fred--,NaN,NaN,NaN,course-v1:ulb+44001+session04,1,0,"Hello everybody, I am Fred and I am living nea...",5,NaN,NaN
2,-BS-,NaN,NaN,NaN,MinesTelecom/04016/session01,0,2,"Bonsoir, \nmême constat. Bonsoir Stéphanie, \n...",33,NaN,NaN
3,-Dominique-,NaN,NaN,NaN,course-v1:ulb+44001+session04,0,1,Hello from Saint Raphael (South France). My na...,0,NaN,NaN
4,-Elo-,NaN,NaN,NaN,course-v1:ulb+44001+session04,1,0,"Hi everybody, I'm Elodie and I'm from Brussel...",2,NaN,NaN


In [ ]:
# data.dropna(subset=['certificate_eligible', 'grade'], inplace = True)

In [3]:
data.describe(include = object)

,user,gender,country,level_of_education,course_id,corpus,certificate_eligible
count,63889,953,599,665,63889,63889,5349
unique,54134,2,37,9,230,63779,2
top,EGo41,m,FR,m,course-v1:AdelaideX+Project101x+2T2016,idem,Y
freq,49,822,491,292,6671,14,3308


In [4]:
data.describe(exclude = object)

,nb_threads,nb_comments,delai_1er_post,grade
count,63889.000000,63889.000000,63889.000000,5349.000000
mean,1.078245,2.342485,33.234860,0.618854
std,1.734308,13.117775,64.004331,0.367677
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,4.000000,0.260000
50%,1.000000,1.000000,14.000000,0.770000
75%,1.000000,2.000000,36.000000,0.950000
max,70.000000,878.000000,1144.000000,1.000000


In [31]:
# plot du nb de moocs par user, devrait détecter EGo41
nan_count = data['user'].isna().sum() #aucun user sans nom

user_count =data["user"].value_counts(ascending=False)

# user_count = user_count.reset_index
user_count

user_count = pd.DataFrame(user_count, columns = ["count","nope"])

user_count = user_count.reset_index()
user_count
user_count.head()



,index,user,count
0,EGo41,49,NaN
1,Moalic1,25,NaN
2,bgaultier,22,NaN
3,ElsaMarcand,21,NaN
4,FlorianeBD,20,NaN


In [ ]:

# data_canada = px.data.gapminder().query("country == 'Canada'")

fig = px.bar(data, x='year', y='pop')
fig.show()